In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm

# Functions

In [2]:
def getRC(mtm, collateral):
    # Replacement Cost (unmargined trades only)
    return max(sum(mtm - collateral), 0)

def getTimeBucket(start_date, end_date):
    diff = end_date - start_date
    if diff < 1:
        return 1
    elif 1 <= diff <= 5:
        return 2
    else:
        return 3

def getAdjustedNotional(start_date, end_date, trade_notional):
    # supervisory duration
    SD = (math.exp(-0.05 * start_date) - math.exp(-0.05 * end_date))/0.05
    return SD*trade_notional

def getSupervisoryDelta(start_date, option_type, pay_leg, price, strike, volatility):
    # For instruments that are not options or CDO tranches
    if option_type == np.nan or start_date == 0:
        if pay_leg == 'Fix':
            return 1  # Long in the primary risk factor
        else:
            return -1  # Short in the primary risk factor
    else:
        t = (np.log(price / strike) + 0.5 * volatility ** 2 * start_date / (volatility * start_date ** (1 / 2)))
        
        # For Call Options
        if option_type == 'Call':
            # is bought
            if pay_leg == 'Fix':
                return norm.cdf(t)
            # is sold
            else:
                return -norm.cdf(t)
            
        # For Put Options
        else:
            # is bought
            if pay_leg == 'Fix':
                return -norm.cdf(-t)
            # is sold
            else:
                return norm.cdf(-t)
            
def getD(start_date, end_date, supervisory_delta, adjusted_notional):
    diff = end_date - start_date
    maturity_factor = (np.minimum(diff, 1) / 1) ** (1 / 2)
    return supervisory_delta * adjusted_notional * maturity_factor

def getEffectiveNotional(D1,D2,D3):
    return math.sqrt(D1 ** 2 + D2 ** 2 + D3 ** 2 + 1.4 * (D1 * D2 + D2 * D3) + 0.6 * D1 * D3)

def getMultiplier(RC, AddOn):
    return np.minimum(1, 0.05 + 0.95 * math.exp(RC / (2 * 0.95 * AddOn)))

# Input

In [3]:
data = pd.read_excel('./Input_Data_Interest_Rate.xlsx')
df = data.copy()

In [4]:
df.sample(5)

,TRADE_ID,PRODUCT,HEDGING_SET,START_DATE,END_DATE,TRADE_NOTIONAL,PAY_LEG,RECEIVE_LEG,OPTION TYPE,PRICE,STRIKE,VOLATILITY,SUPERVISORY FACTOR,MTM,COLLATERAL
294,74,Interest Rate Swap,CAD,0.0,7.4,740000,Fix,Floating,NaN,0.196061,NaN,0.5,0.005,225568.0,224568.0
106,10,Interest Rate Swap,USD,0.0,1.0,100000,Fix,Floating,NaN,0.050000,NaN,0.5,0.005,15520.0,14520.0
119,214,European swaption,EUR,0.0,1.1,2140000,Floating,Fix,PUT,0.055455,0.05,0.5,0.005,685048.0,684048.0
48,88,Interest Rate Swap,CAD,0.0,0.5,880000,Fix,Floating,NaN,0.023000,NaN,0.5,0.005,271516.0,270516.0
55,189,Interest Rate Swap,GBP,0.0,1.8,1890000,Floating,Fix,NaN,0.023485,0.20,0.5,0.005,602998.0,601998.0


# Calculation

In [5]:
# RC
RC = getRC(df["MTM"], df['COLLATERAL'])
# Time Bucket
df['time_bucket'] = df.apply(lambda x:getTimeBucket(x['START_DATE'], x['END_DATE']), axis=1)
# Adjusted Notional
df['adjusted_notional'] = df.apply(lambda x:getAdjustedNotional(x['START_DATE'], x['END_DATE'], x['TRADE_NOTIONAL']), axis=1)
# Supervisory Delta
df['supervisory_delta'] = df.apply(lambda x:getSupervisoryDelta(x['START_DATE'], x['OPTION TYPE'], x['PAY_LEG'], x['PRICE'], x['STRIKE'], x['VOLATILITY']), axis=1)
# D
df['D'] = df.apply(lambda x:getD(x['START_DATE'], x['END_DATE'], x['supervisory_delta'], x['adjusted_notional']), axis=1)
# Effective Notional
EN = []
currency = df['HEDGING_SET'].unique()
for i in currency:
    D1 = df.loc[(df["HEDGING_SET"] == i) & (df["time_bucket"]==1),"adjusted_notional"]
    D2 = df.loc[(df["HEDGING_SET"] == i) & (df["time_bucket"]==2),"adjusted_notional"]
    D3 = df.loc[(df["HEDGING_SET"] == i) & (df["time_bucket"]==3),"adjusted_notional"]

    # effective notional for each time bucket
    D1 = 0 if D1.empty else D1.iloc[0] * df.loc[(df["HEDGING_SET"] == i) & (df["time_bucket"]==1),"supervisory_delta"].iloc[0]
    D2 = 0 if D2.empty else D2.iloc[0] * df.loc[(df["HEDGING_SET"] == i) & (df["time_bucket"]==2),"supervisory_delta"].iloc[0]
    D3 = 0 if D3.empty else D3.iloc[0] * df.loc[(df["HEDGING_SET"] == i) & (df["time_bucket"]==3),"supervisory_delta"].iloc[0]

    en = getEffectiveNotional(D1,D2,D3)
    EN.append(en)
    
# AddOn
AddOn = sum(EN) * 0.005
# multiplier
Multiplier = getMultiplier(RC, AddOn)
# EAD 
alpha = 1.4
EAD = alpha * (RC + Multiplier * AddOn)

# Result

In [6]:
EAD

715588.2014156948